In [ ]:

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
import wget

from langchain_community.llms import LlamaCpp

In [2]:
filename = 'companyPolicies.txt'
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6JDbUb_L3egv_eOkouY71A.txt'

# Use wget to download the file
wget.download(url, out=filename) #url is saved as filename so it creates a new file
print('file downloaded') 

file downloaded


In [3]:
with open(filename, 'r') as file:
    # Read the contents of the file
    contents = file.read()
    print(contents)

1.	Code of Conduct

Our Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.
Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.
Respect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.
Accountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to continuously improve our practices. We report any potential violations of 

Split into chunks

In [ ]:
print(f"Loading '{filename}' using LangChain's TextLoader...")
loader = TextLoader(filename, encoding='utf-8') 
try:
    documents = loader.load() # This line is key. It reads the file and creates a list of Document objects.
    print(f"Successfully loaded {len(documents)} LangChain Document object(s).")
    if documents:
        print(f"Type of 'documents': {type(documents)}")
        print(f"Type of the first item in 'documents': {type(documents[0])}")
        print(f"Preview of first document's page_content (first 100 chars): '{documents[0].page_content[:100]}...'")
    else:
        print("Warning: No documents were loaded by TextLoader. The file might be empty or an issue occurred.")
except Exception as e:
    print(f"Error loading document with TextLoader: {e}")
    documents = [] 

Loading 'companyPolicies.txt' using LangChain's TextLoader...
Successfully loaded 1 LangChain Document object(s).
Type of 'documents': <class 'list'>
Type of the first item in 'documents': <class 'langchain_core.documents.base.Document'>
Preview of first document's page_content (first 100 chars): '1.	Code of Conduct

Our Code of Conduct outlines the fundamental principles and ethical standards th...'


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter 


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,  
    separators=["\n\n", "\n", ". ", " ", ""] 
)
texts = text_splitter.split_documents(documents)
print(f"Split into {len(texts)} chunks.")

Split into 27 chunks.


embeddding and storing

In [6]:
embeddings = HuggingFaceEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)  # store the embedding in docsearch using Chromadb
print('document ingested')

document ingested


In [7]:
from langchain_community.llms import LlamaCpp 

print("Initializing Llama-2-7B-Chat LLM via LlamaCpp...")
model_path = "C:\AI models LLM\llama-2-7b-chat.Q4_K_M.gguf"

llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=0,      # Start with 0 for CPU to ensure it runs.
    n_batch=512,
    n_ctx=4096,
    f16_kv=True,
    temperature=0.5, 
    max_tokens=512,
    verbose=True
)
print("Llama-2-7B-Chat LLM initialized.")



llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from C:\AI models LLM\llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u3

Initializing Llama-2-7B-Chat LLM via LlamaCpp...


llama_model_loader: - kv  14:                  tokenizer.ggml.token_type arr[i32,32000]   = [2, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...
llama_model_loader: - kv  15:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  16:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - kv  17:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  18:               general.quantization_version u32              = 2
llama_model_loader: - type  f32:   65 tensors
llama_model_loader: - type q4_K:  193 tensors
llama_model_loader: - type q6_K:   33 tensors
print_info: file format = GGUF V2
print_info: file type   = Q4_K - Medium
print_info: file size   = 3.80 GiB (4.84 BPW) 
init_tokenizer: initializing tokenizer for type 1
load: control token:      2 '</s>' is not marked as EOG
load: control token:      1 '<s>' is not marked as EOG
load: special_eos_id is not in special_eog_ids - the tokenizer config m

Llama-2-7B-Chat LLM initialized.


CPU : SSE3 = 1 | SSSE3 = 1 | AVX = 1 | AVX2 = 1 | F16C = 1 | FMA = 1 | LLAMAFILE = 1 | OPENMP = 1 | AARCH64_REPACK = 1 | 
Model metadata: {'general.name': 'LLaMA v2', 'general.architecture': 'llama', 'llama.context_length': '4096', 'llama.rope.dimension_count': '128', 'llama.embedding_length': '4096', 'llama.block_count': '32', 'llama.feed_forward_length': '11008', 'llama.attention.head_count': '32', 'tokenizer.ggml.eos_token_id': '2', 'general.file_type': '15', 'llama.attention.head_count_kv': '32', 'llama.attention.layer_norm_rms_epsilon': '0.000001', 'tokenizer.ggml.model': 'llama', 'general.quantization_version': '2', 'tokenizer.ggml.bos_token_id': '1', 'tokenizer.ggml.unknown_token_id': '0'}
Using fallback chat format: llama-2


In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
import uuid # To generate a unique collection name

print("Initializing HuggingFace embeddings...")
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
print(f"Embeddings initialized with model: {embedding_model_name}")


unique_collection_name = f"rag_collection_{uuid.uuid4().hex}"
print(f"Creating Chroma vector store (docsearch) with unique collection name: {unique_collection_name}...")

docsearch = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    collection_name=unique_collection_name, # Explicitly use the unique name
    persist_directory=None # Explicitly ensure it's in-memory
)
print(f"Chroma vector store (docsearch) created with collection: {unique_collection_name}")


print("Creating RetrievalQA chain...")
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    return_source_documents=False
)
print("RetrievalQA chain (qa) ready.")

print("\n--- Querying the RAG system ---")
query = "what is mobile policy?"
print(f"\nQuestion: {query}")

try:
    result = qa.invoke(query)
    if isinstance(result, dict) and "result" in result:
        print("\nAnswer from LLM:")
        print(result["result"])
    elif isinstance(result, str):
        print("\nAnswer from LLM:")
        print(result)
    else:
        print("\nReceived result (type might be unexpected):")
        print(result)
except Exception as e:
    print(f"Error during qa.invoke(query) with direct string: {e}")
    print("Attempting qa.invoke({'query': query}) instead...")
    result_dict = qa.invoke({"query": query})
    print("\nAnswer from LLM (using dict input):")
    if "result" in result_dict:
        print(result_dict["result"])
    else:
        print("Result from dict input (key 'result' not found):")
        print(result_dict)

print("\n--- RAG Querying Finished ---")

Initializing HuggingFace embeddings...
Embeddings initialized with model: sentence-transformers/all-MiniLM-L6-v2
Creating Chroma vector store (docsearch) with unique collection name: rag_collection_c1442de24cc443bb8bedd22c58ee3ab4...
Chroma vector store (docsearch) created with collection: rag_collection_c1442de24cc443bb8bedd22c58ee3ab4
Creating RetrievalQA chain...
RetrievalQA chain (qa) ready.

--- Querying the RAG system ---

Question: what is mobile policy?


llama_perf_context_print:        load time =   14040.39 ms
llama_perf_context_print: prompt eval time =   14039.92 ms /   480 tokens (   29.25 ms per token,    34.19 tokens per second)
llama_perf_context_print:        eval time =   11617.44 ms /   102 runs   (  113.90 ms per token,     8.78 tokens per second)
llama_perf_context_print:       total time =   25718.13 ms /   582 tokens



Answer from LLM:
 Based on the provided context, a mobile policy is a set of guidelines that outline how employees should use their mobile devices in a way that is consistent with company values and legal compliance. It covers various aspects such as acceptable use, security, confidentiality, cost management, compliance, and what to do in case of lost or stolen devices.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

--- RAG Querying Finished ---


In [10]:
query = "what is my name?"
print(f"\nQuestion: {query}")

try:
    result = qa.invoke(query)
    if isinstance(result, dict) and "result" in result:
        print("\nAnswer from LLM:")
        print(result["result"])
    elif isinstance(result, str):
        print("\nAnswer from LLM:")
        print(result)
    else:
        print("\nReceived result (type might be unexpected):")
        print(result)
except Exception as e:
    print(f"Error during qa.invoke(query) with direct string: {e}")
    print("Attempting qa.invoke({'query': query}) instead...")
    result_dict = qa.invoke({"query": query})
    print("\nAnswer from LLM (using dict input):")
    if "result" in result_dict:
        print(result_dict["result"])
    else:
        print("Result from dict input (key 'result' not found):")
        print(result_dict)

print("\n--- RAG Querying Finished ---")


Question: what is my name?


Llama.generate: 45 prefix-match hit, remaining 731 prompt tokens to eval
llama_perf_context_print:        load time =   15236.58 ms
llama_perf_context_print: prompt eval time =   24181.40 ms /   731 tokens (   33.08 ms per token,    30.23 tokens per second)
llama_perf_context_print:        eval time =    4362.88 ms /    37 runs   (  117.92 ms per token,     8.48 tokens per second)
llama_perf_context_print:       total time =   28562.55 ms /   768 tokens



Answer from LLM:
 Unfortunately, I don't have access to any personal information about you, including your name. Therefore, I cannot provide an answer to the question as I do not know your name.

--- RAG Querying Finished ---
